# Uncertainty Analysis with NZUpy's Range Runs

## Overview of Range Runs

NZUpy allows for uncertainty analysis through "range runs," which highlight how uncertainties in the response of gross emissions demand to carbon pricing can affect market outcomes. This notebook demonstrates:

1. How to set up and run 'range runs'
2. Use a custom function to manually edit input data
3. Generate range charts with uncertainty bands
4. Extract and export results using inbuilt NZUpy features

Let's begin by importing the necessary libraries and setting up our model.

## 1. Set Up the NZUpy Model

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

#Set up NZUpy and chart generation
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))
from model.core.base_model import NZUpy
from model.utils.chart_generator import ChartGenerator
from model.interface.chart_display import create_chart_page

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "03_range_runs"
os.makedirs(output_dir, exist_ok=True)

In [2]:
# Initialise NZUpy
NZU = NZUpy(data_dir=data_dir)

## 2. Set Up Range Scenario Type

For uncertainty analysis, we need to set the model to use "Range" scenario type, which automatically configures the model to run multiple scenarios with a span of demand response sensitivities. The rest of the set-up process remains similar to running single scenarios through NZUpy.


In [ ]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

# Set the scenario type to 'Range' for uncertainty analysis
NZU.define_scenario_type('Range')

# Prime the model - this will automatically set up the 5 sensitivity levels
NZU.prime()

## 3. Configure Policy Scenario

We'll set up our Range run to examine a topical policy scenario in which the Government actively pursues reform to supply for industrial allocation and forestry removals. We'll do this by:

1. Loading the Goverment's 'low' projection for forestry removals.
2. Using a custom function to adjust default industrial allocation projections.

In [ ]:
# Our approach to configuring the range run is similar to that for single scenario runs.
NZU.configure_range_scenarios()

First we'll adjust forestry to one of the pre-loaded Government forecasts, the 'low' forecast from the Government's most recent Emissions Reduction Plan.

In [ ]:
# Apply the low forestry config to all sensitivity level scenarios
for i in range(len(NZU.scenarios)):
    NZU.use_config(i, 'forestry', 'low')

# Verify forestry configuration for each sensitivity level
print("\nVerifying forestry configuration for each sensitivity level:")
for i, scenario in enumerate(NZU.scenarios):
    config = NZU.component_configs[i]
    print(f"Scenario {i} ({scenario}) forestry config: {config.forestry}")

Next we'll adjust industrial allocation. First we'll use the `get_industrial_allocation_data` method to extract the default configured industrial allocation volumes we loaded in earlier, then apply a linear reduction through to 2040.

In [ ]:
# Get the current industrial allocation data
industrial_data = NZU.data_handler.get_industrial_allocation_data(config='central')

# Create our phase-out schedule from 2024 (100%) to 2040 (0%)
phase_out_years = range(2024, 2041)
phase_out_factors = np.linspace(1.0, 0.0, len(phase_out_years))

# Create a new DataFrame with phase-out
phased_allocation = industrial_data.copy()

# Apply the phase-out factors
for i, year in enumerate(phase_out_years):
    if year in phased_allocation.index:
        phased_allocation.loc[year, 'baseline_allocation'] = industrial_data.loc[year, 'baseline_allocation'] * phase_out_factors[i]

# Set any years after 2040 to zero
for year in phased_allocation.index:
    if year > 2040:
        phased_allocation.loc[year, 'baseline_allocation'] = 0.0

Next we can load in the custom values using the `set_series` method. As we're doing a *Range* run, we'll need to ensure the custom industrial allocation volumes are applied to each demand sensitivity scenario (i.e., all scenarios in our index). 

In [ ]:
# Apply the industrial allocation phase-out to ALL sensitivity levels
for i in range(len(NZU.scenarios)):
    NZU.set_series('baseline_allocation', phased_allocation['baseline_allocation'], 'industrial', scenario_index=i)

## 4. Run the Model

Now let's run the model with our configured range scenario involving lower afforestation, and phased out industrial allocation by 2040.

In [8]:
# Run the model
results = NZU.run()

## 5. Generate Range Charts

As with our single scenario runs, we can use inbuilt charting functions to quickly produce charts for our range run. Let's explore this capability by generating charts for carbon price, gross emissions demand and stockpile balance. Range run charts will show the varying demand sensitivity scenarios as a band of uncertainty.

In [ ]:
# Initialise chart generator
chart_gen = ChartGenerator(NZU)

# Generate carbon price chart with uncertainty bands
print("Generating carbon price chart with uncertainty bands...")
price_chart = chart_gen.carbon_price_chart()
price_chart.update_layout(title="Carbon Price Projection with Uncertainty Bands")
price_chart.show()
price_chart.write_image(str(output_dir / "range_price_chart.png"))

# Generate emissions pathway chart with uncertainty bands
print("\nGenerating emissions pathway chart with uncertainty bands...")
emissions_chart = chart_gen.emissions_pathway_chart()
emissions_chart.update_layout(title="Emissions Pathway with Uncertainty Bands")
emissions_chart.show()
emissions_chart.write_image(str(output_dir / "range_emissions_chart.png"))

# Generate stockpile balance chart with uncertainty bands
print("\nGenerating stockpile balance chart with uncertainty bands...")
stockpile_chart = chart_gen.stockpile_balance_chart()
stockpile_chart.update_layout(title="Stockpile Balance with Uncertainty Bands")
stockpile_chart.show()
stockpile_chart.write_image(str(output_dir / "range_stockpile_chart.png"))

## 6. Create Interactive HTML Dashboard

NZUpy also provides functionality to create interactive HTML dashboards with multiple charts. Let's create one for our range run, which'll be saved to `\examples\outputs\03_range_runs\range_run_dashboard.html`

In [ ]:
# Create a collection of charts for our dashboard
chart_collection = {
    "Carbon Price Projection": price_chart,
    "Emissions Pathway": emissions_chart,
    "Stockpile Balance": stockpile_chart
}

# Create an interactive HTML dashboard
dashboard_html = create_chart_page(
    charts=chart_collection,
    title="NZUpy Range Run Results",
    subtitle="Uncertainty analysis with industrial allocation phase-out by 2040 and low forestry",
    output_dir=output_dir,
    filename="range_run_dashboard.html"
)

## 7. Extract and Analyse Uncertainty Ranges

Let's also extract the data from our Pandas multi-index dataframe and analyse the range results for some key metrics.

In [ ]:
# Extract carbon price data for all sensitivity levels
price_data = NZU.prices.xs('carbon_price', level='variable', axis=1)

# Focus on a selection of years for each demand sensitivity scenario
forecast_years = [2030, 2035, 2040, 2045, 2050]

print("\nUncertainty ranges for carbon prices ($/tCO2e):")
print(f"{'Year':<10} {'Central':<10} {'±1 Std Err':<15} {'95% Range':<20}")
print("-" * 55)

for year in forecast_years:
    if year in price_data.index:
        central = price_data['central'][year]
        lower_std = price_data['1 s.e lower'][year]
        upper_std = price_data['1 s.e upper'][year]
        lower_95 = price_data['95% Lower'][year]
        upper_95 = price_data['95% Upper'][year]
        
        std_range = f"[${upper_std:.1f} - ${lower_std:.1f}]"
        conf_range = f"[${upper_95:.1f} - ${lower_95:.1f}]"
        
        print(f"{year:<10} ${central:<10.1f} {std_range:<15} {conf_range:<20}")

# Now do the same for emissions
emissions_data = NZU.demand.xs('emissions', level='variable', axis=1)

print("\nUncertainty ranges for emissions (kt CO2e):")
print(f"{'Year':<10} {'Central':<10} {'±1 Std Err':<15} {'95% Range':<20}")
print("-" * 55)

for year in forecast_years:
    if year in emissions_data.index:
        central = emissions_data['central'][year]
        lower_std = emissions_data['1 s.e lower'][year]
        upper_std = emissions_data['1 s.e upper'][year]
        lower_95 = emissions_data['95% Lower'][year]
        upper_95 = emissions_data['95% Upper'][year]
        
        std_range = f"[{upper_std:,.0f} - {lower_std:,.0f}]"
        conf_range = f"[{upper_95:,.0f} - {lower_95:,.0f}]"
        
        print(f"{year:<10} {central:<10,.0f} {std_range:<15} {conf_range:<20}")


As we can see above, varying demand elasticity assumptions can affect price and gross emissions outcomes, with a more elastic demand response seeing lower emissions and a lower carbon price.

## 8. Exporting results data for later use

NZUpy also provides helper functions to explore available variables and export results for further analysis. 

First, let's remind ourselves what variables are available in our results using the helper `list_variables` method:

In [ ]:
# Create an output formatter instance
from model.utils.output_format import OutputFormat
output_formatter = OutputFormat(NZU)

# List all available variables by category
output_formatter.list_variables()

We can examine and export a single variable if we wish. Lets check the head of our data for the ratio of stockpile size to annual demand.

In [ ]:
# Get the stockpile ratio to demand for all scenarios
stockpile_ratio = NZU.stockpile.xs('ratio_to_demand', level='variable', axis=1)

# Display the first few rows to check the data
print("\nStockpile ratio to demand (first & last 5 years):")
print(stockpile_ratio.head(10).round(2))

Interesting, we see a decreasing stockpile size relative to annual demand at most demand sensitivity levels over this time period, though this trend starts to level off and reverse trend from the early 2030s in our higher elasticity scenarios (1 s.e upper & 95% upper). We can export this to a CSV for later use too.

In [14]:
# Create a DataFrame with first 10 years as index and scenarios as columns
export_df = pd.DataFrame(index=stockpile_ratio.index[:10])
export_df.index.name = 'year'  # Name the index column

# Add the stockpile ratio data for each scenario
for scenario in NZU.scenarios:
    export_df[f'stockpile_ratio_{scenario}'] = stockpile_ratio[scenario][:10].round(2)

#Export to CSV
csv_path = output_dir / 'stockpile_ratio_to_annual_demand.csv'
export_df.to_csv(csv_path)

Or we can batch save all our results data to CSV files. Lets place these in a subfolder for safekeeping. 

In [ ]:
# Export all model data to separate CSV files
print("\nExporting all model data to separate CSV files...")
chart_gen.export_csv_data(output_dir=output_dir / 'csv_data')

## 9. Conclusion

This notebook has demonstrated NZUpy's capability to perform uncertainty analysis through range runs. We've seen how to:

1. **Set up a range run**
2. **Configure custom policy settings** with industrial allocation phased-out more quickly and a 'low' forestry configuration used.
3. **Generate and interpret range charts** with uncertainty bands.
4. **Create interactive dashboards**
5. **Extract and export uncertainty range results** for different results variables.

Range runs are a powerful tool, allowing users to understand not just the expected outcomes of different settings but also the uncertainty around those projections. This approach provides a more complete picture for decision-making, but care needs to be taken in interpretation of what the range represents.